# 00B — RBI Macro Data Load & Validation

**Purpose**: Load all RBI CSVs, clean columns, wide→long conversion, identify usable macro series

**Inputs**: `../data_raw/rbi/*.csv`

**Outputs**:
- `../data_processed/rbi_macro_all_long.parquet`
- Coverage table (series × frequency × years)

---

In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Paths
RBI_PATH = Path('../data_raw/rbi')
PROCESSED_PATH = Path('../data_processed')
PROCESSED_PATH.mkdir(exist_ok=True)

print(f"RBI CSV files found: {len(list(RBI_PATH.glob('*.csv')))}")
for f in RBI_PATH.glob('*.csv'):
    print(f"  • {f.name}")

RBI CSV files found: 8
  • Dailyother.csv
  • Macro-economic Indicators - Fortnightly.csv
  • Macro-economic Indicators - Monthly.csv
  • Macro-economic Indicators - Quarterly.csv
  • Macro-economic Indicators - Weekly.csv
  • otherMonthly.csv
  • otherQuarterly.csv
  • otherWeekly.csv


## 1. Define Frequency Mapping

In [2]:
# Map filenames to frequencies
FREQUENCY_MAP = {
    'Dailyother.csv': 'daily',
    'Macro-economic Indicators - Weekly.csv': 'weekly',
    'otherWeekly.csv': 'weekly',
    'Macro-economic Indicators - Fortnightly.csv': 'fortnightly',
    'Macro-economic Indicators - Monthly.csv': 'monthly',
    'otherMonthly.csv': 'monthly',
    'Macro-economic Indicators - Quarterly.csv': 'quarterly',
    'otherQuarterly.csv': 'quarterly'
}

print("Frequency mapping defined")

Frequency mapping defined


## 2. Load and Inspect Each CSV

In [3]:
def load_rbi_csv(filepath: Path) -> pd.DataFrame:
    """Load RBI CSV with flexible date parsing."""
    # Try reading with different encodings
    for encoding in ['utf-8', 'latin-1', 'cp1252']:
        try:
            df = pd.read_csv(filepath, encoding=encoding)
            break
        except:
            continue
    
    return df

# Load and inspect all files
rbi_raw = {}
for csv_file in RBI_PATH.glob('*.csv'):
    try:
        df = load_rbi_csv(csv_file)
        rbi_raw[csv_file.name] = df
        print(f"\n📁 {csv_file.name}")
        print(f"   Shape: {df.shape}")
        print(f"   Columns: {list(df.columns[:5])}{'...' if len(df.columns) > 5 else ''}")
    except Exception as e:
        print(f"✗ {csv_file.name}: {e}")


📁 Dailyother.csv
   Shape: (3922, 8)
   Columns: ['Reporting Date', 'NSE S&P CNX NIFTY', 'BSE BANKEX', 'REPO RATE (OVERNIGHT)', 'REVERSE REPO RATE (OVERNIGHT)']...

📁 Macro-economic Indicators - Fortnightly.csv
   Shape: (217, 10)
   Columns: ['Period', 'Non Food Credit\n (?  Crore)', 'Investment In India\n (? Crore)', 'Aggregate Desposits \n(? Crore)', 'Bank Credit \n(? Crore)']...

📁 Macro-economic Indicators - Monthly.csv
   Shape: (99, 22)
   Columns: ['Period', 'Consumer Price Index  (2012=100)', 'Index of Industrial Production', 'Wholesale Price Index (2011-12=100)', 'Consumer Price Index for Agricultural Labourer']...

📁 Macro-economic Indicators - Quarterly.csv
   Shape: (32, 5)
   Columns: ['Period', 'Overall Balance of Payments Net \n(US $ Million)', 'International Investment Position Net\n(US $ Million)', "India's External Debt - Gross Total \n (US $ Million) ", 'Gross External Debt\nPosition']

📁 Macro-economic Indicators - Weekly.csv
   Shape: (433, 17)
   Columns: ['Peri

## 3. Standardize Column Names & Parse Dates

In [4]:
def identify_date_column(df: pd.DataFrame) -> str:
    """Find the date column in a DataFrame."""
    date_candidates = ['Date', 'date', 'DATE', 'Period', 'period', 'Month', 'Quarter']
    for col in date_candidates:
        if col in df.columns:
            return col
    # Return first column as fallback
    return df.columns[0]

def parse_date_column(series: pd.Series) -> pd.Series:
    """Parse various date formats."""
    # Try multiple date formats
    for fmt in [None, '%d-%m-%Y', '%Y-%m-%d', '%d/%m/%Y', '%b-%Y', '%B %Y', '%Y-%m']:
        try:
            if fmt:
                return pd.to_datetime(series, format=fmt, errors='coerce')
            else:
                return pd.to_datetime(series, dayfirst=True, errors='coerce')
        except:
            continue
    return pd.to_datetime(series, errors='coerce')

def standardize_rbi_df(df: pd.DataFrame, filename: str) -> pd.DataFrame:
    """Standardize column names and set date index."""
    df = df.copy()
    
    # Find and parse date column
    date_col = identify_date_column(df)
    df['Date'] = parse_date_column(df[date_col])
    
    # Drop rows with invalid dates
    df = df.dropna(subset=['Date'])
    df = df.set_index('Date')
    
    # Drop original date column if different
    if date_col != 'Date' and date_col in df.columns:
        df = df.drop(columns=[date_col])
    
    # Clean column names
    df.columns = [str(c).strip().replace('\n', ' ').replace('  ', ' ') for c in df.columns]
    
    return df

# Standardize all DataFrames
rbi_standardized = {}
for filename, df in rbi_raw.items():
    try:
        std_df = standardize_rbi_df(df, filename)
        rbi_standardized[filename] = std_df
        print(f"✓ {filename}: {len(std_df)} rows, {len(std_df.columns)} series")
    except Exception as e:
        print(f"✗ {filename}: {e}")

✓ Dailyother.csv: 3921 rows, 7 series
✓ Macro-economic Indicators - Fortnightly.csv: 217 rows, 9 series
✓ Macro-economic Indicators - Monthly.csv: 99 rows, 21 series
✓ Macro-economic Indicators - Quarterly.csv: 32 rows, 4 series
✓ Macro-economic Indicators - Weekly.csv: 433 rows, 16 series


✓ otherMonthly.csv: 899 rows, 30 series
✓ otherQuarterly.csv: 63 rows, 60 series
✓ otherWeekly.csv: 577 rows, 4 series


## 4. Convert Wide → Long Format

In [5]:
def wide_to_long(df: pd.DataFrame, frequency: str, source_file: str) -> pd.DataFrame:
    """Convert wide format to long format."""
    # Reset index to make Date a column
    df_reset = df.reset_index()
    
    # Melt to long format
    long_df = df_reset.melt(
        id_vars=['Date'],
        var_name='series_name',
        value_name='value'
    )
    
    # Add metadata
    long_df['frequency'] = frequency
    long_df['source_file'] = source_file
    
    # Convert value to numeric
    long_df['value'] = pd.to_numeric(long_df['value'], errors='coerce')
    
    # Drop rows with null values
    long_df = long_df.dropna(subset=['value'])
    
    return long_df

if rbi_standardized:
    # Convert all to long format
    long_dfs = []
    for filename, df in rbi_standardized.items():
        frequency = FREQUENCY_MAP.get(filename, 'unknown')
        long_df = wide_to_long(df, frequency, filename)
        long_dfs.append(long_df)
        print(f"{filename}: {len(long_df)} observations in long format")

    # Combine all
    rbi_long = pd.concat(long_dfs, ignore_index=True)
    print(f"\nTotal observations: {len(rbi_long):,}")
else:
    print("No standardized RBI data to convert")

Dailyother.csv: 20515 observations in long format


Macro-economic Indicators - Fortnightly.csv: 412 observations in long format
Macro-economic Indicators - Monthly.csv: 663 observations in long format
Macro-economic Indicators - Quarterly.csv: 1 observations in long format
Macro-economic Indicators - Weekly.csv: 6249 observations in long format
otherMonthly.csv: 11061 observations in long format
otherQuarterly.csv: 681 observations in long format
otherWeekly.csv: 2302 observations in long format

Total observations: 41,884


## 5. Identify Unique Series & Coverage

In [6]:
if 'rbi_long' in locals():
    # Get unique series
    series_list = rbi_long['series_name'].unique()
    print(f"Unique macro series: {len(series_list)}")

    # Calculate coverage per series
    coverage = rbi_long.groupby(['series_name', 'frequency']).agg(
        start_date=('Date', 'min'),
        end_date=('Date', 'max'),
        observations=('value', 'count'),
        mean_value=('value', 'mean'),
        std_value=('value', 'std')
    ).reset_index()

    # Calculate years of data
    coverage['years'] = (coverage['end_date'] - coverage['start_date']).dt.days / 365.25
    coverage = coverage.sort_values('years', ascending=False)

    print(f"\nSeries with 5+ years of data: {(coverage['years'] >= 5).sum()}")
    print(f"Series with 10+ years of data: {(coverage['years'] >= 10).sum()}")

Unique macro series: 99

Series with 5+ years of data: 88
Series with 10+ years of data: 59


In [7]:
if 'coverage' in locals():
    # Show top series by coverage
    print("Top 20 Series by Data Coverage:")
    print(coverage.head(20)[['series_name', 'frequency', 'start_date', 'end_date', 'years', 'observations']])

Top 20 Series by Data Coverage:
                                          series_name frequency start_date  \
74                   OTHER DEPOSITS WITH RESERVE BANK   monthly 1951-03-01   
48                     DEMAND DEPOSITS WITH THE BANKS   monthly 1951-03-01   
38                           CURRENCY WITH THE PUBLIC   monthly 1951-03-01   
67                                                 M3   monthly 1951-03-01   
65                                                 M1   monthly 1951-03-01   
37  CREDIT TO THE COMMERCIAL SECTOR BY THE BANKING...   monthly 1961-03-01   
68                        MARKET CAPITALISATION - BSE   monthly 1980-03-01   
56                              FOREIGN TRADE IMPORTS   monthly 1990-04-01   
55                              FOREIGN TRADE EXPORTS   monthly 1990-04-01   
66                                                 M2   monthly 1994-03-01   
90  SUM OF CREDIT TO THE GOVERNMENT & CREDIT TO TH...   monthly 1999-04-01   
52                              

## 6. Frequency Distribution

In [8]:
if 'coverage' in locals():
    # Count series by frequency
    freq_summary = coverage.groupby('frequency').agg(
        num_series=('series_name', 'count'),
        avg_years=('years', 'mean'),
        total_obs=('observations', 'sum')
    ).round(1)

    print("Series by Frequency:")
    print(freq_summary)

Series by Frequency:
             num_series  avg_years  total_obs
frequency                                    
daily                 7       10.7      20515
fortnightly           2        7.9        412
monthly              44       24.9      11724
quarterly            27       10.0        682
weekly               19        8.6       8551


## 7. Filter Usable Series (5+ years)

In [9]:
if 'coverage' in locals():
    # Keep only series with 5+ years of data
    usable_series = coverage[coverage['years'] >= 5]['series_name'].tolist()
    rbi_long_filtered = rbi_long[rbi_long['series_name'].isin(usable_series)].copy()

    print(f"Usable series (5+ years): {len(usable_series)}")
    print(f"Filtered observations: {len(rbi_long_filtered):,}")

Usable series (5+ years): 88
Filtered observations: 41,656


## 8. Export to Parquet

In [10]:
if 'rbi_long' in locals():
    # Save full long-format data
    output_path = PROCESSED_PATH / 'rbi_macro_all_long.parquet'
    rbi_long.to_parquet(output_path, index=False)
    print(f"✓ Saved: {output_path}")

if 'coverage' in locals():
    # Save coverage table
    coverage_path = PROCESSED_PATH / 'rbi_macro_coverage.parquet'
    coverage.to_parquet(coverage_path, index=False)
    print(f"✓ Saved: {coverage_path}")

✓ Saved: ..\data_processed\rbi_macro_all_long.parquet
✓ Saved: ..\data_processed\rbi_macro_coverage.parquet
